# COPY INTO and MERGE Commands
  - Incrementally loads data into Delta Lake tables from Cloud Storage.
  - Supports schema evolution.
  - Supports wide range of the file formats (CSV, JSON, Parquet, Delta).
  - Alternative to Auto Loader for batch ingestion.

In [0]:
%sql
CREATE TABLE IF NOT EXISTS demo.delta_lake.raw_stock_prices;

In [0]:
%sql


DELETE FROM demo.delta_lake.raw_stock_prices;


COPY INTO demo.delta_lake.raw_stock_prices
FROM 'abfss://demo@deacourseextdl137.dfs.core.windows.net/landing/stock_prices'
FILEFORMAT = JSON
FORMAT_OPTIONS ('inferSchema' = 'true')
COPY_OPTIONS ('mergeSchema' = 'true');

In [0]:
%sql
SELECT * FROM demo.delta_lake.raw_stock_prices;

In [0]:
%sql
DESC EXTENDED demo.delta_lake.raw_stock_prices;

## MERGE Statement
- Used for upserts (Insert/Update/Delete operations in a single command)
- Allow merging new data into a target table based on matching condition

In [0]:
%sql
CREATE TABLE IF NOT EXISTS demo.delta_lake.stock_prices (
  stock_id STRING,
  price DOUBLE,
  trading_date DATE
);

### Merge the souece data into target table
1. Insert new stocks received.
2. Update price and trading_status if updates recieived.
3. Delete stocks which are de-listed from the exchange (status = 'DELISTED')

In [0]:
%sql
MERGE INTO demo.delta_lake.stock_prices AS target
USING demo.delta_lake.raw_stock_prices AS source
  ON target.stock_id = source.stock_id
WHEN MATCHED AND source.status = 'ACTIVE' THEN
  UPDATE SET target.price = source.price, target.trading_date = source.trading_date
WHEN MATCHED AND source.status = 'DELISTED' THEN
  DELETE
WHEN NOT MATCHED AND source.status = 'ACTIVE' THEN
  INSERT *;


In [0]:
%sql
SELECT * FROM demo.delta_lake.stock_prices;